In [ ]:
# python3 /home/rnb/Projects/rnb-planning/src/pkg/planning/filtering/lattice_model/shared_lattice_predictor.py --rtype panda

# import SharedArray as sa
# ROBOT_TYPE_NAME = 'indy7gripper'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

In [1]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "stowing")
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, "stowing-result")
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_count", "failed_MPs", "IK_count", "failed_IKs", "success"]

# obj1

In [ ]:
RTYPE = "indy7gripper"
DAT_DIR = '20210707-034434'
FILE_OPTION = "obj_1"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
GRASP_SAMPLE = 100
VISUALIZE = False
PLAY_RESULT = False
SHOW_STATE = False

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100


### test

In [ ]:
I_START = 0
N_DAT = 50-I_START

In [ ]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

# obj3

In [6]:
RTYPE = "indy7gripper"
DAT_DIR = '20210707-034434'
FILE_OPTION = "obj_3"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# GRASP_SAMPLE = 100
# VISUALIZE = True
# PLAY_RESULT = True
# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# SHOW_STATE = False


In [7]:
I_START=0
N_DAT = 50-I_START

### test

In [ ]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

------- 5.3     3.8     4.4     6.8     -------
========== 1 / 50 (eta: 20.3 / 1014.4 s) =========
------- 23.8    9.6     5.3     4.7     -------
========== 2 / 50 (eta: 63.6 / 1590.0 s) =========
------- 25.2    8.7     10.7    4.5     -------
========== 3 / 50 (eta: 112.7 / 1878.7 s) =========
------- 25.8    8.4     20.1    104.0   -------
========== 4 / 50 (eta: 271.0 / 3387.3 s) =========
------- 27.4    4.8     8.3     4.6     -------
========== 5 / 50 (eta: 316.0 / 3160.5 s) =========
------- 13.3    4.3     4.7     7.2     -------
========== 6 / 50 (eta: 345.4 / 2878.4 s) =========
------- 33.1    108.0   4.3     8.4     -------
========== 7 / 50 (eta: 499.1 / 3565.1 s) =========
------- 18.7    4.2     5.7     6.5     -------
========== 8 / 50 (eta: 534.1 / 3338.4 s) =========
------- 103.7   5.1     4.7     5.2     -------
========== 9 / 50 (eta: 652.8 / 3626.7 s) =========
------- 6.5     5.0     4.4     5.3     -------
========== 10 / 50 (eta: 674.0 / 3369.9 s) =========
-

# obj3_pole

In [3]:
RTYPE = "indy7gripper"
DAT_DIR = '20210707-034434'
# DAT_DIR = '20210702-152658'
FILE_OPTION = "obj_3_pole"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3_pole"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# GRASP_SAMPLE = 100
# VISUALIZE = True
# PLAY_RESULT = True
# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# SHOW_STATE = False


In [ ]:
I_START=0
N_DAT = 50-I_START

### test

In [ ]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')
gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

## Res - panda
```

```

## Res - indy7
```
============================== obj_1           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 18.58           6.56            4.06            0.11            2               20             
Tool           : 6.96            6.12            4.0             0.18            3               20             
ToolReach      : 2.79            5.44            3.67            0.17            2               20             
Full           : 2.22            5.25            3.12            0.0             4               20             
 
============================== obj_3           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 30.37           8.73            5.2             0.07            5               20             
Tool           : 9.77            5.27            5.47            0.07            5               20             
ToolReach      : 7.24            6.0             7.42            0.17            8               20             
Full           : 6.2             5.53            4.53            0.0             5               20             
 
============================== obj_3_pole      ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 45.17           8.36            5.07            0.21            6               20             
Tool           : 16.48           5.71            7.65            0.53            3               20             
ToolReach      : 18.59           6.23            10.46           0.69            7               20             
Full           : 11.28           5.29            5.14            0.14            6               20             
```